In [ ]:
# Import required libraries
import geopandas as gpd
import os
from joblib import Parallel, delayed

# Define input parameters
# Define input paths (Replace these with actual paths)
flood_folder = r"./data/flood_scenarios"  # Folder containing flood shapefiles
output_folder = r"./output/closed_roads"  # Folder to save extracted road data
road_shapefile = r"./data/network/road_network.shp"  # Road network shapefile

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Define filename pattern for flood files
base_filename = "flood_scenario_"

# Load road shapefile
road_gdf = gpd.read_file(road_shapefile)

# Function to Process Each Flood Shapefile
def process_flood_file(index):
    suffix = f"{index:02d}"  # Format index as two-digit number
    flood_file = f"{base_filename}{suffix}.shp"
    flood_path = os.path.join(flood_folder, flood_file)

    if not os.path.exists(flood_path):
        print(f"File not found: {flood_file}")
        return None

    try:
        # Load flood data
        flood_gdf = gpd.read_file(flood_path)

        # Filter flood geometries with water depth ≥ 0.3m
        filtered_flood = flood_gdf[flood_gdf['water_depth'] >= 0.3]
        if filtered_flood.empty:
            print(f"No valid geometries in {flood_file}")
            return None

        # Ensure CRS alignment
        filtered_flood = filtered_flood.to_crs(road_gdf.crs)

        # Find intersecting roads
        intersecting_roads = road_gdf[road_gdf.intersects(filtered_flood.unary_union)]
        road_ids = intersecting_roads['road_id'].tolist()

        # Save road IDs to a text file
        output_file = os.path.join(output_folder, f"{flood_file.replace('.shp', '')}_intersecting_roads.txt")
        with open(output_file, "w") as file:
            for road_id in road_ids:
                file.write(f"{road_id}\n")

        print(f"Processed {flood_file}: {len(road_ids)} roads saved.")
        return len(road_ids)
    except Exception as e:
        print(f"Error processing {flood_file}: {e}")
        return None
# Process files in parallel
# Define the range of file indices (e.g., 73 flood scenarios)
file_indices = range(73)

# Parallel processing with Joblib
results = Parallel(n_jobs=-1)(delayed(process_flood_file)(index) for index in file_indices)

# Summary of processing
total_processed = sum(1 for result in results if result is not None)
print(f"Total files processed: {total_processed}")

